<img src="./images/Front_Lightning.svg"/>

[Photo](https://www.flickr.com/photos/ikewinski/8029865920) by Mike Lewinski / [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)

In [36]:
# Note: run but don't show this block.
# Stubs so the examples works
class DeathRay(object):
    def __init__(self, *args, **kwargs): pass
    def vaporize(self, *args, **kwargs): pass
class TimeMachine(object):
    def go(self, *args, **kwargs):
        pass

# The Cyborg class

In [37]:
import time

class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

This is our class, and we want to log the calls to _every_ method.

# Attempt 1: print() calls everywhere

No point in denying it: this is what many of us _would_ do if not supervised by an adult. 

In [38]:
import time

class Cyborg(object):

    def __init__(self, name):
        print("Creating new Cyborg with name '{}'".format(name))
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        print("Travelling to {} and year {}".format(destination, year))
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        print("Attacking {}".format(target))
        self.weapon.vaporize(target)

In [39]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

Creating new Cyborg with name 'T-1000'
Travelling to Los Angeles and year 1995
Attacking Sarah Connor
Attacking John Connor


"The most effective debugging tool is still careful thought, coupled with judiciously placed print statements"  [Brian W. Kernighan](https://en.wikipedia.org/wiki/Brian_Kernighan).

# Attempt 2: The `logging` module

In [40]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)

In [41]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-23 13:07:35,306 Creating new Cyborg with name 'T-1000'
2017-09-23 13:07:35,308 Travelling to Los Angeles and year 1995
2017-09-23 13:07:35,559 Attacking Sarah Connor
2017-09-23 13:07:35,560 Attacking John Connor


# Problem: there're logging calls everywhere

- We must add manually the call to e.g. `logging.info()` to every method.
- Whoever adds a new method in the future might forget to do it.
- What if we had three _hundred_ methods, instead of three?
- Also, copy-pasting function calls is boring.

In [42]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()
        self.alive = True

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)
        
    def selfdestroy(self):
        self.alive = False

We added `Cyborg.selfdestroy()`, but forgot to add the logging call.

# Detour: Decorators Crash Course

What are decorators and what are they useful for?

<img src="./images/Lightning_Łódź.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/paszczak000/3659732549/) by Kamil Porembiński / [CC BY-SA 2.0](https://creativecommons.org/licenses/by-sa/2.0/)

# First-class objects

> "A first class object is an entity that can be dynamically created, destroyed, passed to a function, returned as a value, and have all the rights as other variables in the programming language have" --- [StackOverflow](https://stackoverflow.com/a/245208)

* Functions are first-class objects.
* Classes are first-class objects too.
* In fact, in Python _everything_ is a first-class object.

This mean that if we take e.g. a function, we can...


## Assign it to a variable

Take a function, for example. We can assign it to a variable like any other object.

In [43]:
numbers = [3, 5, 7, 1]
print(max(numbers))

7


In [44]:
func = max
print(func(numbers))  # same as calling max()

7


No parentheses because we are not calling the function.

## Pass it as argument

... _to_ another function:

In [45]:
def call(func, values):
    return func(values)

print(call(max, numbers))
print(call(min, numbers))

7
1


This is used in real life for [callbacks][1], among others.

[1]: https://en.wikipedia.org/wiki/Callback_(computer_programming)

## Return it

... _from_ another function:

In [46]:
import random                                                               

func = random.choice([max, min])
print("Function:", func)
print("Result:", func(numbers))

Function: <built-in function max>
Result: 7


## Nested functions

Define it within another function, and return it!

In [47]:
def get_power_function(exponent):
    """Returns a function to compute the exponent-th power."""
    
    def power(n):
        return n ** exponent
    return power

square = get_power_function(2)
cube   = get_power_function(3)
n = 4

print("Number:", 4)
print("Square:", square(n))
print("Cube  :", cube(n))

Number: 4
Square: 16
Cube  : 64


# So, decorators...

... are "wrappers" that let us execute code _before_ and _after_ the function that they decorate without modifying the function itself. We:

* Take the function as an argument.
* Add some behaviour, wrapping it in a new function.
* Return (and later use) the new function.

In [48]:
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    def wrapped(*args):
        tstart = time.time()
        result = func(*args)
        tend = time.time()
        tdelta = tend - tstart
        print("Function call took {} seconds".format(tdelta))
        return result
    
    return wrapped

Note: no `**kwargs` for simplicity's sake.

A more Pythonic approach, by the way, would be to use [finally](https://docs.python.org/3/tutorial/errors.html#defining-clean-up-actions):

In [49]:
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    def wrapped(*args):
        try:
            tstart = time.time()
            return func(*args)
        finally:
            tend = time.time()
            tdelta = tend - tstart
            print("Function call took {} seconds".format(tdelta))

    return wrapped

No need to store the result in a variable to return it later.

Let's now decorate something:

In [50]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

func = measure_time(square_everything)
print(func(numbers))

Function call took 3.337860107421875e-06 seconds
[9, 25, 49, 1]


# Missing attributes

A problem with our decorated function is that we lose attributes such as the name or docstring.

In [51]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

print("Name:", square_everything.__name__)
print("Docstring:", square_everything.__doc__)

Name: square_everything
Docstring: Return the square of all the numbers.


However...

In [52]:
func = measure_time(square_everything)

print("Name:", func.__name__)
print("Docstring:", func.__doc__)

Name: wrapped
Docstring: None


This is unfortunate, as these are great for e.g. debugging.

# functools.wraps()

The [`wraps()`](https://docs.python.org/3/library/functools.html#functools.wraps) function allows us to overwrite the function attributes (`__name__`, `__doc__`, `__module__`, etc) attributes of the wrapper function with those of the _original_ function.

In [53]:
import functools
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    @functools.wraps(func)  # note this
    def wrapped(*args):
        try:
            tstart = time.time()
            return func(*args)
        finally:
            tend = time.time()
            tdelta = tend - tstart
            print("Function call took {} seconds".format(tdelta))

    return wrapped

In this manner, the changes to the function are transparent.

In [54]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

func = measure_time(square_everything)

print("Name:", func.__name__)
print("Docstring:", func.__doc__)

Name: square_everything
Docstring: Return the square of all the numbers.


# Python's Decorator Syntax

But we don't want to call our function `func`.

In [55]:
func = measure_time(square_everything)

Let's keep the original name:

In [56]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

square_everything = measure_time(square_everything)
print(square_everything(numbers))

Function call took 4.0531158447265625e-06 seconds
[9, 25, 49, 1]


Instead of...

In [57]:
square_everything = measure_time(square_everything)

... we can apply a decorator using this shortcut:

In [58]:
@measure_time
def square_everything(numbers):
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

print(square_everything(numbers))

Function call took 3.0994415283203125e-06 seconds
[9, 25, 49, 1]


That is: apply `measure_time` to `square_everything` and store it in `square_everything`. We're effective replacing it with the updated, wrapped version.

# There's much more

* We can chain decorators, applying 2+ to the same function.
* We can have decorators that take arguments. 
* A closely-related concept are _closures_.

But this is enough for now.

### Recommended readings:

* [Decorator Basics](https://stackoverflow.com/a/1594484) on Stack Overflow.
* [The closures that moved Spielberg](https://www.youtube.com/watch?v=rrL3CQNOFRc) at PyConES 2016.

# Going back to our problem...

<img src="./images/Lightning_Manila.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/sumarieslabber/33375841246/) by Sumarie Slabber / [CC BY-ND 2.0](https://creativecommons.org/licenses/by-nd/2.0/)

We left it here, with our logging calls _everywhere_.

In [59]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()
        self.alive = True

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)

# Attempt 3: Method decorators

In [60]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapped(*args):  # omit **kwargs for simplicity
        logging.info('Called %s, args=%s', func.__name__, args)
        return func(*args)
    return wrapped

@log
def cube(n):
    """Return the third power of 'n'."""
    return n ** 3

print(cube(2))

2017-09-23 13:07:36,207 Called cube, args=(2,)


8


So going back to our `Cyborg` class...

In [61]:
import time
import logging

def log(func):
    @functools.wraps(func)
    def wrapped(*args):  # omit **kwargs for simplicity
        # args[1:] so that we don't print 'self'
        logging.info('Called %s%s', func.__name__, args[1:])
        return func(*args)
    return wrapped


class Cyborg(object):
    
    @log
    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    @log
    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    @log
    def attack(self, target):
        self.weapon.vaporize(target)

In [62]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-23 13:07:36,286 Called __init__('T-1000',)
2017-09-23 13:07:36,287 Called travel('Los Angeles', 1995)
2017-09-23 13:07:36,538 Called attack('Sarah Connor',)
2017-09-23 13:07:36,539 Called attack('John Connor',)


# This didn't solve the problem, though

- We have replaced calls to `logging.info()` with calls to `@log`.
- Whoever adds a new method in the future might forget to _decorate_ it.
- We still could have three _hundred_ methods instead of three.
- Also, copy-pasting method decorations is still boring.

# Attempt 4: Class decorators

Let a second decorator do the work for us $\rightarrow$ decorate all the methods.



In [63]:
def decorate_all_methods(decorator):
    def wrapped(cls):
        for attr_name in cls.__dict__:
            attr = getattr(cls, attr_name)
            if callable(attr):
                # It's a function, decorate it
                setattr(cls, attr_name, decorator(attr))
        return cls
    return wrapped

In [64]:
@decorate_all_methods(log)
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

In [65]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-23 13:07:36,619 Called __init__('T-1000',)
2017-09-23 13:07:36,621 Called travel('Los Angeles', 1995)
2017-09-23 13:07:36,874 Called attack('Sarah Connor',)
2017-09-23 13:07:36,875 Called attack('John Connor',)


# We did it! Hoo-ray!

In [66]:
@decorate_all_methods(log)
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

# We _didn't_ solve the problem, though

- We just moved one level up the ladder of abstraction.
- We have replaced decorating methods with decorating _classes_.
- Whoever adds a new _class_ in the future might forget to _decorate_ it.
- What if we had one _hundred_ classes instead of just one?

In [67]:
@decorate_all_methods(log)
class Ninja(object): pass
    # ...

@decorate_all_methods(log)
class Human(object): pass
    # ...

@decorate_all_methods(log)
class Terminator(object): pass
    # ...  

# Nested classes

Another limitation is that out decorator will also decorate _nested_ classes, even if that's not what we want.

In [68]:
@decorate_all_methods(log)
class Cyborg(object):

    # This class was also decorated.
    class Chainsaw(object):
        
        def vaporize(self, victim): pass
            # ...            
    
    def __init__(self, name):
        self.name = name
        self.weapon = Cyborg.Chainsaw()

    def attack(self, target):
        self.weapon.vaporize(target)

robot = Cyborg('T-1000')
robot.attack('Sarah Connor')

2017-09-23 13:07:36,992 Called __init__('T-1000',)
2017-09-23 13:07:36,993 Called Chainsaw()
2017-09-23 13:07:36,994 Called attack('Sarah Connor',)


# There are more limitations

For example, if we want to be able to add _classes_:

In [69]:
Android = Human + Robot  # get new class and use it
replicant = Android('Roy Batty')

NameError: name 'Robot' is not defined

### Recommended readings:

* [Python metaclasses vs class decorators](https://stackoverflow.com/a/1779404) on Stack Overflow.
* [What are Python metaclasses useful for?](https://stackoverflow.com/a/1779404) also by Alex Martelli.

There're things that simply _cannot_ be done with class decorators.

But _metaclasses_ can.

# Help me, Obi-Wan Kenobi. You're my only hope

<img src="./images/Lightning_Storm.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/texaus1/33124985460/) by texaus1 / [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)

# Classes and instances

* Every object is the instance of a class.
* We can check this with the built-in [`type()`](https://docs.python.org/3/library/functions.html#type).

In [70]:
number = 21
word = "blue"

print(type(number))
print(type(word))

<class 'int'>
<class 'str'>


Let's check our own class too:

In [73]:
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

arnold = Cyborg("T-800")
print(type(arnold))

<class '__main__.Cyborg'>


# Wait, but...

* (1) _Every_ object is an instance of a class.
* (2) We said that classes are _objects_ too.
* From (1) and (2) it follows that classes are instances of... what?

In [74]:
print(type(int))
print(type(str))
print(type(Cyborg))

<class 'type'>
<class 'type'>
<class 'type'>


So all classes are instances of class `type`.

# Down the rabbit hole

And `type` is an instance of...

In [75]:
print(type(type))

<class 'type'>


Everything is an instance of `type`, including _itself_.

# The Most Important Slide

If you're going to remember only one thing, it should be this:

> Instances of a class are to **classes** what classes are to **metaclasses**.

<img src="./images/instance-of.png" style="height: 400px; width:auto"/>

* In the same way a class defines the creation and behaviour of an instance of that class...
* ... metaclasses allow us to define the creation and behaviour of our **classes**.

# `type()` to define new classes

As we just saw, the built-in [`type()`](https://docs.python.org/3/library/functions.html#type) returns the type on an object.

In [76]:
numbers = [1, 2, 3]
print(type(numbers))

<class 'list'>


However, it can also be called with _three_ arguments to return a **new type** object. [From the docs](https://docs.python.org/3/library/functions.html#type):

> With three arguments, return a new type object. This is essentially a dynamic form of the [`class`](https://docs.python.org/3/reference/compound_stmts.html#class) statement. The *name* string is the class name and becomes the [`__name__`](https://docs.python.org/3/library/stdtypes.html#definition.__name__) attribute; the *bases* tuple itemizes the base classes and becomes the [`__bases__`](https://docs.python.org/3/library/stdtypes.html#class.__bases__) attribute; and the *dict* dictionary is the namespace containing definitions for class body and is copied to a standard dictionary to become the [`__dict__`](https://docs.python.org/3/library/stdtypes.html#object.__dict__) attribute. For example, the following two statements create identical [`type`](https://docs.python.org/3/library/functions.html#type) objects:


In [77]:
class X(object):
    a = 1
    
# The above class is identical to:
X = type('X', (object,), dict(a=1))

# Another example

In [78]:
name = "Cylon"
base = (object,)

# We need 'self', of course.
def attack(self, victims):
    print("Destroy all {}!".format(victims))

body = {'attack': attack}
    
Cylon = type(name, base, body)
centurion = Cylon()
centurion.attack("humans")

Destroy all humans!


# How is this useful?

What's the purpose of creating classes dynamically with `type()`?

* We definitely don't need them _often_.
* However, _sometimes_ is the appropriate solution.
* For example, for GUI programming it's very convenient to define classes on the fly and instantiate widgets with them.

## A real-life example: [SQLAlchemy](https://www.sqlalchemy.org/)

> There is only one way to register a database table with SQLAlchemy: create a `Model` class describing the table (not unlike Django's models). To get SQLAlchemy to recognize a table, a class for that table must be created in some way. Since `sandman` doesn't have any advanced knowledge of the database structure, it can't rely on pre-made model classes to register tables. Rather, it needs to introspect the database and create these classes on the fly. Sound familiar? **Any time you're creating new classes dynamically, `type` is the correct/only choice**.

[Improve Your Python: Metaclasses and Dynamic Classes With Type](https://jeffknupp.com/blog/2013/12/28/improve-your-python-metaclasses-and-dynamic-classes-with-type/), by Jeff Knupp.

# Metaclasses — finally!

It took us a while, but we're finally here.

<img src="./images/Lightning_Cagliari.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/berlinrider/14878240502/) by Chris / [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)

# The silliest metaclass ever

In [79]:
class Meta42(type):
    
    def __new__(meta, name, bases, kwargs):
        return 42

class Cyborg42(metaclass=Meta42):
    pass

print(Cyborg42)

42


Woahhhh! What happened here? As you can see, the '__new__' method of the metaclass is in charge of creating the class itself. In this example we simply return a number, which is totally allowed even if the number is not itself a class. Python trusts us — it should't.

# The yolooo metaclass


In [82]:
def make_yoloooo(func):
    
    def wrapper(*args,**kwargs):
        try:
            return func(*args,**kwargs)
        except Exception as ex:
            print("{} supressed".format(ex.__class__.__name__))
    return wrapper


class MetaYoloooo(type):
    
    def __new__(meta, base, names, kwargs):
        for name,attr in kwargs.items():
            if callable(attr):
                kwargs[name] = make_yoloooo(attr)
        return super().__new__(meta,base,names,kwargs)

In this example, we are creating a decorator that will supress all exceptions raised in the decorated function. To make it work we only need to create a metaclass that automatically decorates all methods inside a class. 

At this point we only need to use our new metaclass like this:

In [86]:
class DefectiveCyborg(object, metaclass=MetaYoloooo):

    def __init__(self, name):
        raise ValueError

    def travel(self, destination, year):
        raise ZeroDivisionError

    def attack(self, target):
        raise RuntimeError

robot = DefectiveCyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

ValueError supressed
ZeroDivisionError supressed
RuntimeError supressed
RuntimeError supressed


# The registering metaclass


Now for some real_life™ examples. If we are designing some library that allows the user to declare its own classes based on the ones we provide (imagine for example Django, SQLAlchemy, Luigy, Ansible, ...etc) and we want to perform some kind of operations based on the classes that the user has defined along with some of our own design. It would be very handy if we can have automatically a registry of all the clases that have been declared using the ones we provide. This is very easy with metaclasses:

In [88]:
import types

class MetaRegister(type):

    _registry = dict()

    def __new__(meta, name, bases, kwargs):
        kwargs["_registry"] = types.MappingProxyType(meta._registry)
        cls = super().__new__(meta,name,bases,kwargs)
        meta._registry[name] = cls
        return cls

We can now define the base class our library will provide:

In [ ]:
class Cyborg(object, metaclass=MetaRegister):
    pass

The user of our library will declare some classes based on this last one:

In [ ]:
class Terminator(Cyborg):
    pass

class Ninja(object):
    pass

class CyborgNinja(Cyborg, Ninja):
    pass

Now we have all the clases available as a read-only attribute inside every class and as a read-write attribute inside the metaclass:

In [ ]:
print(MetaRegister._registry) 
print(CyborgNinja._registry)

Note that we can only modify the metaclass registry as the ones inside the regular classes are read-only:

In [ ]:
# We can edit MetaRegister._registry:
MetaRegister._registry["tuple"] = tuple
# But now CyborgNinja._registry:
CyborgNinja._registry["list"] = list

# The type enforcing metaclass


Let's play a bit with some dark magic. We can start declaring the following descriptor that checks if the value assigned to it is positive:

In [ ]:
class Positive:
    def __set_name__(self,owner,name):
        self.name = name
    def __get__(self,instance,owner):
        return instance.__dict__[self.name]
    def __set__(self,instance, value):
        if value > 0:
            instance.__dict__[self.name] = value
        else:
            raise ValueError("The value is not positive")

We can use this descriptor in the usual way:

In [ ]:
class Wallet(object):
    money  = Positive()

my_wallet = Wallet()
my_wallet.money = 5
print(my_wallet.money)
my_wallet.money = -5

Python 3 has this awesome feature called "type annotations" that allows us to indicate some typing information about the variables, methods, classes and functions. This is usually not enforced, but we can play a bit with that. Let's define the following metaclass:

In [ ]:
import collections
class MetaAnnotations(type):
    @classmethod
    def __prepare__(metacls, name, base, **kwargs):
        return collections.ChainMap({},{"enforce_positive":Positive})

    def __new__(meta,name,bases,kwargs):
        return super().__new__(meta,name,bases,kwargs.maps[0])

    def __init__(cls,name,bases,kwargs):
        for name,val in getattr(cls,"__annotations__",{}).items():
            desc = val()
            desc.__set_name__(cls,name)
            setattr(cls,name,desc)

We can now define our wallet again with type annotations:

In [ ]:
class Wallet(object,metaclass=MetaAnnotations):
    money  : enforce_positive

my_wallet = Wallet()
my_wallet.money = 5

In [ ]:
print(my_wallet.money)
my_wallet.money = -5

Notice that we also achieve something really weird: "enforce_positive" is not defined anywere as a variable! Our metaclass is allowing us to use the name 'enforce_positive' as an alias for the 'Positive' without needing to import the name into our scope. Awesome!

We can check that using a regular variable in our class definition:

In [ ]:
class Wallet(object,metaclass=MetaAnnotations):
    money = enforce_positive

my_wallet = Wallet()
print(my_wallet.money)

# Abstract Base Clases

Some times is very handy to pay a visit to the old good C++ world and bring some abstract base classes as a souvenir. These are very handy if we want to define interfaces or if we want to enforce some common properties when the users of our library will extend and define their own classes based on the ones we provide. Python make this very easy for us with the 'abc' library:

In [ ]:
from abc import ABCMeta, abstractmethod

class MetaCyborg(metaclass=ABCMeta):
    @abstractmethod
    def travel(self, destination, year):
        ...

    @abstractmethod
    def attack(self, target):
        ...

Once we have defined our abstract base class we can use it like this:

In [ ]:
class PrototypeCyborg(MetaCyborg):
    def __init__(self, name):
        self.name = name

    def travel(self, destination, year):
        print("Traveling really far")

class Cyborg(MetaCyborg):

    def __init__(self, name):
        self.name = name

    def travel(self, destination, year):
        print("Traveling really far")

    def attack(self,target):
        print("Atacking really hard")

In [ ]:
robot = Cyborg('T-1000')

In [ ]:
#This doesn't work
robot = PrototypeCyborg('T-1000')

# No more __init__

Tired of writing the same piece of code thousands of times to declare '__init__'and assign the variables inside? Don't worry because we have exactly the stupid idea you need! The 'NoMoreInit' metaclass:

In [ ]:
import collections
class MetaNoMoreInit(type):
    @classmethod
    def __prepare__(metacls, name, base, **kwargs):
        return collections.OrderedDict()
    def __new__(meta,name,bases,kwargs):
        if "__init__" not in kwargs:
            annotations = kwargs["__annotations__"]
            def __init__(self,*args,**kwargs):
                if args or kwargs.keys() != annotations.keys():
                    raise ValueError("Incorrect arguments, my lord!")
                self.__dict__.update(kwargs)
            kwargs["__init__"] = __init__
        return super().__new__(meta,name,bases,kwargs)

We can now use type annotations (of course) to declare the variables that our class will define and we will have a '__init__' prepared for us:

In [ ]:
class Easy(metaclass=MetaNoMoreInit):
    x:int
    y:float
    z:str

myobj = Easy(x=1,y=3.4,z="Hello")
print(myobj.x)
print(myobj.y)
print(myobj.z)

# Back to our problem

<img src="./images/Lightning_New_York.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/quintanomedia/9334838960/) by Anthony Quintano / [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)

In our pursue for purity and maximum generalization we look back at what we have achieved and we are pleased. But suddenly....something is wrong. Something is still verbose. Something is still......explicit:

In [ ]:
# See this line? It's... explicit
class HappyCyborg(object,metaclass = MetaRegister): 
    ...

We still need to write `metaclass = MyMetaclass` every time we declare a new base class. We don't need to do that in every child class that inherits from it but we still need to do that with the parents. It wouldn't be great if we had a way to avoid that?

# Enter the  `__build_class__` hook

Every time python defines a new class, before doing anything else (even before resolving the metaclasses) it executes a function. This function is part of the Cpython interpreter (is written in C) and is in charge of building the class attribute dictionary among other things. We can actually substitute itś behaviour thanks to a hook that the interpreter exposes for us.

In [ ]:
import builtins
print(builtins.__build_class__) # <--- This is the hook

We an use this to automatically inject out metaclass in the `metaclass` attribute of every class that is ever defined:

In [ ]:
__builtins_build_class = builtins.__build_class__

def _create_custom_build(metaclass):
    def _custom_build(func, name, *bases, **kwargs):
        print([base is not object and isinstance(base, type) for base in bases])
        print(bases)
        if not any((base is not object and isinstance(base, type) for base in bases)):
            if 'metaclass' in kwargs:
                pass
            else:
                kwargs['metaclass'] = metaclass

        return __builtins_build_class(func, name, *bases, **kwargs)
    return _custom_build


To use it we can substitute the original `__build_class__`:

In [ ]:
builtins.__build_class__ = _create_custom_build(lambda *a,**k:42)

class A(object):
    pass

class AHappyAndNormalClass(A):
    pass

print(AHappyAndNormalClass)

We can return everything to normal reasigning the old `__build_class__` back.

In [ ]:
builtins.__build_class__ = __builtins_build_class

# A last crazy idea

We are happy, we are fulfilled. Everything is now stupidly implicit and that makes us great programmers. But after a while we notice something....something that is wrong. **We still need to write out hook in every program we write!**. That makes us sad. It wouldn't be great if we had a way to avoid that? 

# Enter the  `*.pth` files

Python has this thing called `*.pth` files. The deal with them is the following: if python finds a .pth file when in `lib/pythonX.Y/site-packages` that starts with `import` it will **EXECUTE** the file. 

>A path configuration file is a file whose name has the form name.pth and exists in one of the four directories mentioned above; its contents are additional items (one per line) to be added to sys.path. Non-existing items are never added to sys.path, and no check is made that the item refers to a directory rather than a file. No item is added to sys.path more than once. Blank lines and lines beginning with # are skipped. Lines starting with import (followed by space or tab) are executed.



So we can create a `*.pth` file that performs the same operations as our `__build_class__` hook and it will be automatically executed every time we initialize the interpreter. 

❯ pip -V
pip 9.0.1 from /usr/lib/python3.6/site-packages (python 3.6)

❯ cat /usr/lib/python3.6/site-packages/metathing.pth
```python
import metathing
```

❯ cat /usr/lib/python3.6/site-packages/metathing.py

```python
class Meta42(type):
    def __new__(meta,name,bases,kwargs):
        return 42
        
__builtins_build_class = builtins.__build_class__

def _create_custom_build(metaclass):
    def _custom_build(func, name, *bases, **kwargs):
        if not any((base is not object and isinstance(base, type) for base in bases)):
            if 'metaclass' in kwargs:
                pass
            else:
                kwargs['metaclass'] = metaclass

        return __builtins_build_class(func, name, *bases, **kwargs)
    return _custom_build
builtins.__build_class__ = _create_custom_build(Meta42)

```

# Parting Words

> Metaclasses are deeper magic than 99% of users should ever worry about. **If you wonder whether you need them, you don't** (the people who actually need them know with certainty that they need them, and don't need an explanation about why) — [Tim Peters](https://stackoverflow.com/a/2005894).

We may never _need_ them — but now we _know_ what they are.

<img src="./images/Lightning_G0DeX.jpg" style="height: 400px; width:auto"/>

[Photo](https://www.flickr.com/photos/132655238@N06/29110938761/) by G0DeX / [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/0)